In [38]:
import numpy as np

# 5. 오차 역전파법

앞 장에선 기울기를 수치 미분을 사용해 구했다. (해석학적 미분이 안되니까.) 하지만 이는 시간이 많이 걸린다. 

이번엔 가중치 매개변수 기울기를 효율적으로 계산하는 오차 역전파법 (Backpropagation)을 배워보자. 

이를 수치, 그래프 두 가지 방법으로 배울 것이다. 

## 5.1 계산 그래프 (Computational Graph)

그래프 자료구조. node와 edge로 구성됨. 

### 5.1.1 계산 그래프로 풀다. 

<img src="http://img1.daumcdn.net/thumb/R1920x0/?fname=http%3A%2F%2Fcfile26.uf.tistory.com%2Fimage%2F997ED34B5B98F5F235B597">

노드는 연산자를 담당하는 것을 알 수 있다. 계산 그래프를 그리고 왼쪽 --> 오른쪽으로 진행한다. 이것을 '순전파'(forward propagation)라고 부른다. 

반대 방향인 오른쪽 --> 왼쪽으로 가는 것이 '역전파' (backward propagation)이다. 

### 5.1.2 국소적 계산

계산 그래프에선 각 노드의 국소적 계산이 합쳐져 전체의 계산이 이뤄진다. 

즉, 각 노드는 자신과 관련된 계산만 신경쓰면 되고 그 이전의 복잡한 계산은 신경쓸 필요가 없다는 것이다. 

### 5.1.3 왜 계산 그래프로 푸는가? 

계산 그래프의 이점:

1. 국소적 계산. 복잡한 문제를 단순화 할 수 있음. 
2. 중간 계산 결과를 모두 보관할 수 있음. 

그보다 중요한 것은, 

3. 역전파를 통해 미분을 효율적으로 계산할 수 있기 때문. 

역전파를 통해 국소적 미분을 계속 뒤로 전달한다. 

<img src="http://img1.daumcdn.net/thumb/R1920x0/?fname=http%3A%2F%2Fcfile1.uf.tistory.com%2Fimage%2F997E914D5B98F628261EF4">

붉은 색은 dy/dx 이다. 즉, 매우 작은 dx가 증가할 때 y는 그의 1.1배 증가하는 것이다. (그래서 검은 색과 다름.) 

## 5.2 연쇄법칙

역전파가 국소적 미분을 전달하는 원리는 chain rule 이다. 

### 5.2.1 계산 그래프의 역전파 

<img src="http://img1.daumcdn.net/thumb/R1920x0/?fname=http%3A%2F%2Fcfile8.uf.tistory.com%2Fimage%2F999FD3425B98F63F1A9DDF">

신호 E에 노드의 국소적 미분 (ay/ax)를 곱하고 다음 노드로 전달하는 것. 

왜 가능한지 연쇄법칙을 통해 살펴보자. 

### 5.2.2 연쇄법칙이란? 

합성함수는 연쇄법칙이다. 

### 5.2.3 연쇄법칙과 계산 그래프. 

<img src="http://img1.daumcdn.net/thumb/R1920x0/?fname=http%3A%2F%2Fcfile27.uf.tistory.com%2Fimage%2F997387465B98F65D133C64">

이 두 과정이 같은 것이다. 즉, 연쇄법칙을 이용하여 한 단계씩 '국소적 미분'을 해 주는 것이다. 

## 5.3 역전파

### 5.3.1 덧셈 노드의 역전파

덧셈 노드의 역전파는 상류의 값을 그대로 흘려보낸다. 

상류에서 전달받은 aL/az 가 그대로 aL/az * 1이 된다. (왜냐하면 z = x+y 일 때 az/ax = 1 & az/ay = 1이 되므로.)

### 5.3.2 곱셈 노드의 역전파

곱셈 노드의 역전파는 상류의 값에 순전파 때의 입력 신호들을 서로 바꾼 값을 곱해서 하류로 보낸다. 

그림으로 보면 

<img src="http://img1.daumcdn.net/thumb/R1920x0/?fname=http%3A%2F%2Fcfile1.uf.tistory.com%2Fimage%2F99E3EF435B98F69309175D">

상류에서 aL/az가 내려오면 원래 입력신호가 x였던 edge에는 (aL/az)*(az/ax)가 되어야 하니 z = xy 에서 d(xy)/d(x) = y이다. 

따라서 (aL/az) * y 가 된다. 


덧셈의 역전파에선 상류의 값을 그대로 보내므로 순방향 입력 신호가 필요하지 않았으나, 곱셉의 역전파에선 순방향 입력 신호의 값이 필요하다. 

따라서 곱셈 노드를 구현할 때는 순전파의 입력 신호를 변수에 저장해둬야 한다. 

### 5.3.3. 사과 쇼핑의 예

생략. 

## 5.4 단순한 계층 구현하기. 

이제 구현해보자. 

- 덧셈 노드: AddLayer
- 곱셈 노드: MulLayer 

계층을 클래스로 구현할 것이다. 계층이란 신경망의 기능 단위이다. 

### 5.4.1 곱셈 계층. 

모든 계층은 forward()와 backward() method를 가진다. 

In [2]:
class MulLayer:
    def __init__(self): # 초기화, 역전파 때 쓰기 위해 변수 유지. 
        self.x = None
        self.y = None
    
    def forward(self, x, y):
        self.x = x
        self.y = y
        out = x * y
        
        return out
    
    def backward(self, dout):
        dx = dout * self.y # x와 y를 서로 바꿔 곱해야 하니까.
        dy = dout * self.x 
        
        return dx, dy

In [4]:
# 이를 통해 순전파를 구현
apple = 100
apple_num = 2
tax = 1.1

# 계층들
mul_apple_layer = MulLayer() # 연산자 노드. 
mul_tax_layer = MulLayer()

# 순전파
apple_price = mul_apple_layer.forward(apple, apple_num)
price = mul_tax_layer.forward(apple_price, tax)

price

220.00000000000003

In [6]:
# 역전파 구현

dprice = 1
dapple_price, dtax = mul_tax_layer.backward(dprice)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)

print(dapple, dapple_num, dtax)

2.2 110.00000000000001 200


### 5.4.2 덧셈 계층 

In [7]:
class AddLayer:
    def __init__(self): # 변수 저장 안하니 초기화가 필요하지 않다. 
        pass
    
    def forward(self, x, y):
        out = x + y
        return out
    
    def backward(self, dout):
        dx = dout * 1
        dy = dout * 1
        
        return dx, dy

덧셈노드는 곱셈노드보다 쉽다. 

오렌지와 사과 계산 그래프 구현은 생략한다. 

## 5.5 활성화 함수 계층 구현하기. 

계산 그래프를 신경망에 적용하자. 

우선 ReLU와 Sigmoid를 구현한다. 

### 5.5.1 Relu 계층 

Relu 함수는 0보다 크면 x, 작으면 0이기 때문에 미분하면 1, 0 이다. 

따라서 역전파시 순전파 때의 입력 신호가 0보다 컸으면 상류의 값을 그대로 하류로 흘리고 (E * 1), 0보다 작았으면 하류로 신호를 보내지 않음. (E * 0)

<img src="http://img1.daumcdn.net/thumb/R1920x0/?fname=http%3A%2F%2Fcfile28.uf.tistory.com%2Fimage%2F99E517485B98F6E504DB20">

.forward(), .backword()는 numpy 배열을 인수로 받는다. 

* ReLU는 전기 회로 스위치에 비유할 수 있다. 순전파 때 전기가 흐르고 있으면 스위치를 ON으로 하고 흐르지 않으면 OFF로 하기 때문이다. 

In [8]:
class Relu():
    def __init__(self):
        self.mask = None
    
    def forward(self, x):
        self.mask = (x <= 0)
        out = x.copy()
        out[self.mask] = 0
        
        return out
    
    def backward(self, dout):
        dout[self.mask] = 0 # 순전파 때 만들어놓은 mask를 사용. 
        dx = dout
        
        return dx

### 5.5.2 Sigmoid 계층 

<img src="http://img1.daumcdn.net/thumb/R1920x0/?fname=http%3A%2F%2Fcfile27.uf.tistory.com%2Fimage%2F999E3B4B5B98F72021956C">

/ 와 exp() 노드가 보인다. 하지만 원리는 같다. 

1. 우선 y=1/x 를 미분하면 = -(1/x^2) = -y^2 가 된다. 따라서 역전파에선 상류의 값에 원래 입력신호를 제곱하고 음수로 만들어준 값을 곱해 하류로 전달한다.  
2. +1 하는 것은 + 노드니까 상류의 값을 그대로 흘린다. 
3. y = exp()를 미분하면 그대로 = exp() = y 이다. 따라서 상류의 값에 그대로 입력신호를 곱해 하류로 보낸다. 
4. * 노드는 순전파 값을 바꿔 곱한다. 

이렇게 하면 Sigmoid 계층의 역전파 계산 그래프가 완성된다. 

이렇게 계산하면 최종 출력은 aL/ay * (y^2 * exp(-x)) 가 된다. 따라서 Sigmoid 출력은 위의 복잡한 과정 없이 결과적으로 y^2 * exp(-x) 만 있으면 된다.

즉, Sigmoid의 역전파는 순전파의 출력만으로 계산 가능하다. (여기에 상류의 값을 곱하기만 하면 됨.) 

<img src="https://postfiles.pstatic.net/MjAxNzA3MjZfMjc1/MDAxNTAxMDU0ODU2OTYx.u4WNKk87hzczec3x6iZlJXiWxExbaIOlF__qCMO09qkg.UGCo9VPCtL6-IzWhUS0-2ThezrUKP_XfTi-IzoYmxCYg.PNG.cjswo9207/e_5.12.png?type=w1">

이를 하나의 Sigmoid node로 만든다. 노드를 그룹화하여 간소화시키는 것이다. (위의 그림 참조)

이를 구현해보자. 

In [9]:
class Sigmoid:
    def __init__(self):
        self.out = None
        
    def forward(self, x):
        out = 1 / (1 + np.exp(-x))
        self.out = out
        
        return out
    
    def backward(self, dout):
        dx = dout * self.out * (1.0 - self.out) # 순전파 때 저장해놓은 입력 신호를 역전파 때 사용함. 
        
        return dx

<h1 style='color:purple'> 여기서부터 사실 잘 이해가 안된다. 행렬이라 그런가. </h1>

## Affine/Softmax 계층 구현하기. 

### 5.6.1 Affine 게층 

Affine 계층이란? (신경망의 순전파 때 수행하는) 행렬의 곱은 기하학에선 Affine Transformation이라고 부름. 

행렬 곱 node를 dot이라 하자. 이제 스칼라 값 대신 행렬이 흐른다. 

<img src="http://img1.daumcdn.net/thumb/R1920x0/?fname=http%3A%2F%2Fcfile27.uf.tistory.com%2Fimage%2F994002375B98F73E0590F4">

행렬에서 미분을 하면 그림의 1,2와 같은 식이 나온다. 

형상에 주의하자. (행렬 곱을 위해)

### 5.6.2 배치용 Affine 계층

X 하나만이 아닌, N개의 데이터를 묶어(batch) forward 하는 경우를 생각해보자. 

In [12]:
X_dot_W = np.array([[0,0,0], [10,10,10]])
B = np.array([1,2,3])

print(X_dot_W)
print(X_dot_W + B)

[[ 0  0  0]
 [10 10 10]]
[[ 1  2  3]
 [11 12 13]]


In [13]:
dY = np.array([[1,2,3], [4,5,6]])
print(dY)

dB = np.sum(dY, axis=0)
print(dB)

[[1 2 3]
 [4 5 6]]
[5 7 9]


In [15]:
class Affine:
    def __init__(self, W, b):
        self.W = W
        self.b = b
        self.X = None
        self.dW = None
        self.db = None
    
    def forward(self, x):
        self.x = x
        out = np.dot(x, self.W) + self.b
        
        return out
    
    def backward(self, dout):
        dx = np.dot(dout, self.W.T)
        self.dW = np.dot(self.x.T, dout)
        self.db = np.sum(dout, axis=0)

<h1 style='color:purple'>자세한 설명 읽어보기 (부록)</h1>

### 5.6.3 Softmax-with-Loss 계층

출력층의 Softmax는 입력 값을 정규화하여 출력한다. 

신경망에서는 크게 학습, 추론 두 가지를 하는데 

Softmax는 일반적으로 학습할 때 필요하고 추론할 땐 사용되지 않는다. (추론은 결과만 알면 되니까 굳이 확률로 정규화 해 줄 필요는 없다. 그냥 score 높은 것만 보면 되기 때문이다.)

<img src='http://img1.daumcdn.net/thumb/R1920x0/?fname=http%3A%2F%2Fcfile9.uf.tistory.com%2Fimage%2F995A16395B98F76820FB40'>

여기에 Loss Function인 교차 엔트로피 오차도 포함한 Softmax-with-Loss를 구현할 수 있다. 

<img src='http://img1.daumcdn.net/thumb/R1920x0/?fname=http%3A%2F%2Fcfile29.uf.tistory.com%2Fimage%2F99EBF5395B98F7792B42CE'> 

자세한 설명은 부록A에 나와있다. 

그림을 다시 보면, 일단 이 그림은 3 클래스 분류를 가정한 것이다. 

- Softmax는 (a1, a2, a3)를 받아 정규화하여 (y1, y2, y3)를 출력했다.  
- Cross Entropy Error는 (y1, y2, y3)와 정답 레이블 (t1, t2, t3)을 받아 손실 L을 출력했다. 

이 때, 굵은 화살표로 표시된 역전파의 결과를 보자. (y1 - t1, y2 - t2, y3 - t3) 와 같이 깔끔하게 나온다. 

(이는 Softmax에 일부러 Loss function으로 Cross Entropy Error을 썼기 때문임. 회귀의 경우 항등함수에 일부러 Loss function으로 MSE를 쓰면 똑같이 나옴.)

신경망 역전파에선 이 차이가(오차가) 앞 계층에 전해지는 것이다. 

신경망의 학습 목적은 신경망 출력(Softmax 출력)이 정답 레이블과 가까워지도록 가중치 매개변수를 조정하는 것이기 때문에 오차를 잘 전달해야 한다.

Softmax-with-Loss를 구현해보자. 

In [39]:
def softmax(a):
    exp_a = np.exp(a)
    sum_exp_a = np.sum(exp_a)
    y = exp_a / sum_exp_a
    
    return y

In [40]:
def cross_entropy_error(y, t):
    if y.ndim == 1:
        t = t.reshape(1, t.size)
        y = y.reshape(1, y.size)
        
    batch_size = y.shape[0]
    return -np.sum(np.log(np.arange(batch_size), t)) / batch_size

In [19]:
class SoftmaxWithLoss:
    def __init__(self):
        self.loss = None
        self.y = None
        self.t = None
        
    def forward(self, x, t):
        self.t = t
        self.y = softmax(x)
        self.loss = cross_entropy_error(self.y, self.t)
        
        return self.loss
    
    def backward(self, dout=1):
        batch_size = self.t.shape[0]
        dx = (self.y - self.t) / batch_size # batch_size로 나눠 데이터 1개당 오차를 앞 계층으로 전파. 
        
        return dx
        

## 5.7 오차 역전파법 구현하기. 

### 5.7.1 신경망 학습의 전체 그림. 

신경망에는 적응 가능한 가중치와 편향이 있고, 이 가중치와 편향을 훈련 데이터에 적응하도록 조정하는 과정을 '학습'이라 부른다. 이는 총 4단계로 이뤄진다. 

1. 미니배치

훈련 데이터 중 일부를 무작위로 가져온다. 이 선별 데이터를 미니 배치라 하며, 이 미니배치의 손실 함수 값을 줄이는 것이 목표이다. 

2. 기울기 산출

미니배치의 손실 함수 값을 줄이기 위해 각 가중치 매개변수의 기울기를 구한다. 기울기는 손실 함수의 값을 가장 작게하는 방향을 제시한다. (수치미분 --> 오차 역전파법)

3. 매개변수 갱신

가중치 매개변수를 기울기 방향으로 아주 조금 갱신한다. 

4. 반복

1~3을 반복한다. 

### 5.7.2 오차 역전파법을 적용한 신경망 구현하기. 

TwoLayerClass로 구현하겠다. 

TwoLayerClass의 인스턴스 변수
- params: 딕셔너리 변수. 신경망의 매개변수를 보관. (eg: params['W1'])
- layers: 순서 있는 딕셔너리 변수. 신경망의 계층을 보관. (eg: layers['Affine1'], layers['Relu1'])
- lastLayer: 신경망의 마지막 계층. 여기선 SoftmaxWithLoss를 사용. 

TwoLayerClass의 method
- init(): 초기화를 수행
    - input_size: 입력층 뉴런 수
    - hidden_size: 은닉층 뉴런 수
    - output_size: 출력층 뉴런 수 
    - weight_init_std: 가중치 초기화 시 정규분포의 스케일
- predict(self, x): 예측(추론) 수행. x는 이미지 데이터
- loss(self, x, t): 손실함수. x는 이미지, t는 정답 레이블. 
- accuracy(self, x, t): 정확도 구함. 
- numerical_gradient(self, x, t): 가중치 매개변수의 기울기를 수치 미분 방식으로 구함. 
- gradient(self, x, t): 가중치 매개변수 기울기를 오차 역전파법으로 구함. 



In [41]:
import sys, os
sys.path.append(os.pardir)

from common.layers import *
from common.gradient import numerical_gradient
from collections import OrderedDict

import numpy as np

In [42]:
class TwoLayerNet:
    def __init__(self, input_size, hidden_size, output_size, weight_init_std=0.01): # 가중치 초기화. 
        self.params = {}
        self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size)
        self.params['b1'] = np.zeros(hidden_size)
        self.params['W2'] = weight_init_std * np.random.randn(hidden_size, output_size)
        self.params['b2'] = np.zeros(output_size)
        
        # 계층 생성
        self.layers = OrderedDict()
        self.layers['Affine1'] = Affine(self.params['W1'], self.params['b1'])
        self.layers['Relu1'] = Relu()
        self.layers['Affine2'] = Affine(self.params['W2'], self.params['b2'])
        
        self.lastLayer = SoftmaxWithLoss()
        
    def predict(self, x):
        for layer in self.layers.values(): # layers의 원래 순서대로 forward. 
            x = layer.forward(x)
            
        return x 
    
    def loss(self, x, t):
        y = self.predict(x)
        
        return self.lastLayer.forward(y,t)
    
    def accuracy(self, x, t):
        y = self.predict(x)
        y = np.argmax(y, axis=1)
        if t.ndim != 1: t = np.argmax(t, axis=1)
        
        accuracy = np.sum(y==t) / float(x.shape[0])
        
        return accuracy
    
    def numerical_gradient(self, x, t):
        loss_W = lambda W: self.loss(x,t)
        
        grads = {}
        grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
        grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
        grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
        grads['b2'] = numerical_gradient(loss_W, self.params['b2'])
        
        return grads
    
    def gradient(self, x, t):
        # 순전파
        self.loss(x, t)
        
        # 역전파
        dout = 1
        dout = self.lastLayer.backward(dout)
        
        layers = list(self.layers.values())
        layers.reverse() # layers를 반대로 거슬러 올라감. 
        for layer in layers:
            dout = layer.backward(dout)
            
        grads = {}
        grads['W1'] = self.layers['Affine1'].dW
        grads['b1'] = self.layers['Affine1'].db
        grads['W2'] = self.layers['Affine1'].dW
        grads['b2'] = self.layers['Affine1'].db
        
        return grads

순전파 때는 OrderedList인 layers를 그대로, 역전파 때는 거꾸로 reverse 해 하나씩 처리해주면 된다. 

신경망 구성 요소를 계층으로 모듈화하여 구현했기 때문에 쉽게 구현할 수 있었다. 

앞으로도 딥러닝을 할 때 계층을 더 쌓고 싶으면 그냥 이런 방식으로 붙이기만 하면 된다. 

### 5.7.3 오차역전파법으로 구한 기울기 검증하기. 

수치 미분과 해석적 방법을 배웠다. 오차역전파법은 후자에 속하기 때문에 매개변수가 많아도 빠르게 구할 수 있었다. (수치 미분은 느림.)

하지만 수치 미분은 오차역전파법이 제대로 구현되었는지 검증하는 절차에 필요하다. 

오차역전파법은 빠르지만 복잡해 오류가 있기 십상이다. 수치 미분은 느리지만 쉽기 때문에 검증하기 좋다. 

둘을 비교하는 것을 gradient check라고 부른다. 

In [44]:
import sys, os
sys.path.append(os.pardir)
import numpy as np
from dataset.mnist import load_mnist
from two_layer_net import TwoLayerNet

In [51]:
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)

network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

x_batch = x_train[:3]
t_batch = t_train[:3]

grad_numerical = network.numerical_gradient(x_batch, t_batch)
grad_backprop = network.gradient(x_batch, t_batch)

# 각 가중치의 차이의 절대값을 구하고 그 절대값들의 평균을 낸다. 

for key in grad_numerical.keys():
    diff = np.average(np.abs(grad_backprop[key] - grad_numerical[key]))
    print(key + ":" + str(diff))

W1:4.3224236962084585e-10
b1:2.4612101147495357e-09
W2:4.990290551891457e-09
b2:1.402678573861338e-07


위의 오차를 보면 기울기 차이가 거의 없는 것을 볼 수 있다. 검증시 오차가 너무 크면 의심해봐야 한다. 

### 5.7.4 오차역전파법을 사용한 학습 구현하기. 

기울기를 오차역전파법으로 구한다는 점만 다르다. 

In [53]:
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)

network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

iters_num = 10000
train_size = x_train.shape[0]
batch_size = 100
learning_rate = 0.1

train_loss_list = []
train_acc_list = []
test_acc_list = []

iter_per_epoch = max(train_size / batch_size, 1)

for i in range(iters_num):
    batch_mask = np.random.choice(train_size, batch_size)
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]
    
    # 오차 역전파법으로 기울기를 구한다. 
    grad = network.gradient(x_batch, t_batch)
    
    # 갱신
    for key in ('W1', "b1", 'W2', 'b2'):
        network.params[key] = network.params[key] - learning_rate * grad[key]
        
    loss = network.loss(x_batch, t_batch)
    train_loss_list.append(loss)
    
    if i % iter_per_epoch == 0:
        train_acc = network.accuracy(x_train, t_train)
        test_acc = network.accuracy(x_test, t_test)
        train_acc_list.append(train_acc)
        test_acc_list.append(test_acc)
        print(train_acc, test_acc)

0.08516666666666667 0.0813
0.08516666666666667 0.0813
0.08516666666666667 0.0813
0.08516666666666667 0.0813
0.08516666666666667 0.0813
0.08516666666666667 0.0813
0.08516666666666667 0.0813
0.08516666666666667 0.0813
0.08516666666666667 0.0813
0.08516666666666667 0.0813
0.08516666666666667 0.0813
0.08516666666666667 0.0813
0.08516666666666667 0.0813
0.08516666666666667 0.0813
0.08516666666666667 0.0813
0.08516666666666667 0.0813
0.08516666666666667 0.0813


## 5.8 정리

- 계산 그래프를 이용하면 계산 과정을 시각적으로 파악할 수 있다. 
- 계산 그래프의 노드는 국소적 계산으로 구성된다. 국소적 계산을 조합해 전체 계산을 구성한다. 
- 계산 그래프의 순전파는 통상의 계산을 수행한다. 한편, 계산 그래프의 역전파로는 각 노드의 미분을 구할 수 있다. 
- 신경망의 구성 요소를 계층으로 구현하여 기울기를 효율적으로 계산할 수 있다. (오차역전파법)
- 수치 미분과 오차역전파법의 결과를 비교하면 오차역전파법의 구현에 잘못이 없는지 확인할 수 있다. (기울기 확인) 